In [1]:
#General
import numpy as np
import pandas as pd

#Data Retrieval and Visualization
import pandas_datareader.data as web
import datetime
import matplotlib.pyplot as plt
%matplotlib inline

#Statistical Package
from scipy import stats 
import statsmodels.api as sm
import math
from scipy.stats import linregress
from scipy.stats import f

/opt/anaconda3/lib/python3.7/site-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [2]:
from pandas_datareader.famafrench import get_available_datasets
#spy_etf=web.DataReader('SPY','yahoo','2010-01-01','2020-01-01' ) 
beg1='1963-07-01'
end1='2014-12-01' #'2014-12-01'

In [3]:
def fm_summary(p): #Summary Statistics Function
    s = p.describe().T
    s['std_error'] = s['std']/np.sqrt(s['count'])
    s['tstat'] = s['mean']/s['std_error']
    return s[['mean','std','tstat']].T
def update_withrf(dataport):
    for x in range(len(dataport.columns)):
        dataport.iloc[:,x]=dataport.iloc[:,x]-dataport['RF']
    dataport.drop('RF',axis=1,inplace=True)
        

In [4]:
len(get_available_datasets())
#print(ds['DESCR'])
datasets = get_available_datasets()
datasets

['F-F_Research_Data_Factors',
 'F-F_Research_Data_Factors_weekly',
 'F-F_Research_Data_Factors_daily',
 'F-F_Research_Data_5_Factors_2x3',
 'F-F_Research_Data_5_Factors_2x3_daily',
 'Portfolios_Formed_on_ME',
 'Portfolios_Formed_on_ME_Wout_Div',
 'Portfolios_Formed_on_ME_Daily',
 'Portfolios_Formed_on_BE-ME',
 'Portfolios_Formed_on_BE-ME_Wout_Div',
 'Portfolios_Formed_on_BE-ME_Daily',
 'Portfolios_Formed_on_OP',
 'Portfolios_Formed_on_OP_Wout_Div',
 'Portfolios_Formed_on_OP_Daily',
 'Portfolios_Formed_on_INV',
 'Portfolios_Formed_on_INV_Wout_Div',
 'Portfolios_Formed_on_INV_Daily',
 '6_Portfolios_2x3',
 '6_Portfolios_2x3_Wout_Div',
 '6_Portfolios_2x3_weekly',
 '6_Portfolios_2x3_daily',
 '25_Portfolios_5x5',
 '25_Portfolios_5x5_Wout_Div',
 '25_Portfolios_5x5_Daily',
 '100_Portfolios_10x10',
 '100_Portfolios_10x10_Wout_Div',
 '100_Portfolios_10x10_Daily',
 '6_Portfolios_ME_OP_2x3',
 '6_Portfolios_ME_OP_2x3_Wout_Div',
 '6_Portfolios_ME_OP_2x3_daily',
 '25_Portfolios_ME_OP_5x5',
 '25_Portf

In [5]:
#Extracting data
betaPort=web.DataReader('25_Portfolios_ME_BETA_5x5','famafrench',start=beg1,end=end1)
niPort=web.DataReader('25_Portfolios_ME_NI_5x5','famafrench',start=beg1,end=end1)
varPort=web.DataReader('25_Portfolios_ME_VAR_5x5','famafrench',start=beg1,end=end1)
rVarPort=web.DataReader('25_Portfolios_ME_RESVAR_5x5','famafrench',start=beg1,end=end1)
acPort=web.DataReader('25_Portfolios_ME_AC_5x5','famafrench',start=beg1,end=end1)
momPort=web.DataReader('25_Portfolios_ME_Prior_12_2','famafrench',start=beg1,end=end1)
FF5=web.DataReader('F-F_Research_Data_5_Factors_2x3','famafrench',start=beg1,end=end1)
MOM=web.DataReader('F-F_Momentum_Factor','famafrench',start=beg1,end=end1)


In [6]:
betaPort

{0:          SMALL LoBETA  ME1 BETA2  ME1 BETA3  ME1 BETA4  SMALL HiBETA  \
 Date                                                                   
 1963-07       -0.6156    -0.5950     0.1464    -0.4538       -1.0076   
 1963-08        1.2546     4.0686     3.6955     2.0609        3.2983   
 1963-09       -0.8082    -2.0279     0.0034    -1.5411       -3.2312   
 1963-10       -0.2628     1.5439     0.2554     4.2260        2.5346   
 1963-11        0.6858    -2.6122    -0.5701    -1.4607       -1.6136   
 ...               ...        ...        ...        ...           ...   
 2014-08        3.6847     4.0526     4.4137     2.9171        3.6693   
 2014-09       -3.6194    -5.0385    -6.1376    -6.8540       -9.8558   
 2014-10        5.9265     7.5963     6.8120     5.1069        1.9413   
 2014-11       -0.9760    -1.5549    -0.6512    -0.8763       -1.0257   
 2014-12        5.8356     4.0965     5.3248     3.0467        3.3079   
 
          ME2 BETA1  ME2 BETA2  ME2 BETA3  ME2

In [7]:
FF5[0]
FF6=pd.concat([FF5[0],MOM[0]],axis=1,sort=False)

In [8]:
FF6.columns

Index(['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA', 'RF', 'Mom   '], dtype='object')

## Table 1

In [9]:
Table1=fm_summary(FF6).copy()
Table1.rename(str.upper,axis='columns',inplace=True)
Table1.drop('RF',axis=1)

,MKT-RF,SMB,HML,RMW,CMA,MOM
mean,0.507573,0.264110,0.377670,0.267087,0.321715,0.686828
std,4.456559,3.048250,2.818658,2.231847,2.019452,4.220849
tstat,2.831346,2.153915,3.330920,2.974974,3.960338,4.045225


## Table 2 

In [10]:
#Excess return of each anomaly portfolio
betaPortXs = betaPort[0].sub(FF5[0]['RF'],axis=0)
niPortXs = niPort[0].sub(FF5[0]['RF'],axis=0)
varPortXs = varPort[0].sub(FF5[0]['RF'],axis=0)
rVarPortXs = rVarPort[0].sub(FF5[0]['RF'],axis=0)
acPortXs = acPort[0].sub(FF5[0]['RF'],axis=0)
momPortXs = momPort[0].sub(FF5[0]['RF'],axis=0)


In [11]:
def GRS(R,F): #GRS Statistics
    R=R.astype('float64')
    F=F.astype('float64')
    alphas = []
    alpha2=[]
    alphasquared=[]
    errors = []
    betas=[]
    returns=[]
    risquared=[]
    rsquared=[]
    bsealphas=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alphas.append(fit.params["const"])
        alpha2.append(abs(fit.params["const"]))
        alphasquared.append(abs(fit.params["const"])**2)
        risquared.append((R[:][p].mean()-F['Mkt-RF'].mean())**2)
        betas.append(fit.params[1:])
        errors.append(fit.resid)
        returns.append(R[:][p].mean()-F['Mkt-RF'].mean())
        
        rsquared.append(fit.rsquared_adj)
        bsealphas.append(fit.bse["const"]**2)
   #For remaining statistics to compute mean. Convert list to arrasy
    alpha2=pd.np.array(alpha2)
    alphasquared=pd.np.array(alphasquared)
    returns=pd.np.array(returns)
    rsquared=pd.np.array(rsquared)
    bsealphas=pd.np.array(bsealphas)
    risquared=pd.np.array(risquared)
    #For GRS
    alphas = pd.np.array(alphas)
    errors = pd.np.array(errors).transpose()
    Ri=returns.mean()
    K = F.shape[1]
    T, N = errors.shape

    Ve = errors.transpose().dot(errors) / (T - K - 1)#unbiased estimate of the covariance matrix of residuals

    r = F.mean().values #step 5Factor mean vector

    f = F.values

    Vf = (f-r).transpose().dot(f-r) / (T - 1) #Step 7 unbiased estimate of the covariance matrix of the factors

    Sh2_a = alphas.dot(pd.np.linalg.pinv(Ve)).dot(alphas)
    Sh2_f = r.dot(pd.np.linalg.pinv(Vf)).dot(r)
    Sh2_a, Sh2_f
    ans=(T/N)*((T-N-K)/(T-K-1))*(Sh2_a/(1+Sh2_f))
    ans=ans.astype(float)
    #print("P-value of the GRS statistics: ",(1-f.cdf(ans,N,T-N-K)))
    #print("Average Absolute value of intercept A|ai|: ",alpha2.mean())
    #print("Average Absolute value of intercept/return A|ai|/A|ri| : ",alpha2.mean()/Ri)
    #print("Average Absolute value of intercept/return squared: ",(alpha2.mean()**2/Ri**2))
    #print("Average Absolute value of the As^2(ai)/A(ai)^2: ",(bsealphas.mean()))
    #print("Average value of the regression R^2 corrected for degress of freedom: ",rsquared.mean())
    return (ans,N,T-N-K,alpha2.mean(),alpha2.mean()/Ri,(alphasquared.mean()/risquared.mean()),(bsealphas.mean()/alphasquared.mean()),rsquared.mean())

In [12]:
#Factors
F1=FF5[0][['Mkt-RF']] #For portfolio beta only
F2=FF5[0][['Mkt-RF','SMB','HML']]
F3=FF5[0][['Mkt-RF','SMB','HML','RMW']]
F4=FF5[0][['Mkt-RF','SMB','HML','CMA']]
F5=FF5[0][['Mkt-RF','SMB','RMW','CMA']]
F6=FF5[0][['Mkt-RF','SMB','HML','RMW','CMA']]
#For Momentum Anomaly only
F71=FF6[['Mkt-RF','SMB','HML','Mom   ']]
F72=FF6[['Mkt-RF','SMB','HML','RMW','Mom   ']]
F73=FF6[['Mkt-RF','SMB','HML','CMA','Mom   ']]
F74=FF6[['Mkt-RF','SMB','RMW','CMA','Mom   ']]
F75=FF6[['Mkt-RF','SMB','HML','RMW','CMA','Mom   ']]

In [13]:
result=GRS(betaPortXs,F1)
#1-f.cdf(GRS(betaPortXs,F1)[0],GRS(betaPortXs,F1)[1],GRS(betaPortXs,F1)[2])

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [14]:
#Index name for GRS-Statistics
def indexname(f):
    name=""
    for data in list(f.columns):
        name=name+str(data)+" "
    return name
#Initialize
factorList=[F1,F2,F3,F4,F5,F6]
momfactorlist=[F2,F3,F4,F5,F6,F71,F72,F73,F74,F75]
portfolioList=[betaPortXs,niPortXs,varPortXs,rVarPortXs,acPortXs]
portnameList=["35 Size-NI Portffolio","25 Size-Var Portfolio","25 Size-RVar portfolios","25 Size-AC portfolios"] 
column_names=["Model Factors" , "GRS","p(GRS)" ,"A|ai|" ,"A|ai|/A|ri|","Aai^2/Ari^2","As^2(ai)/Aasi^2","A(R^2)"]
df=pd.DataFrame(columns=column_names)
counter=0

In [15]:
df.loc[0]=['25 Size Beta Portfolio',"","","","","","",""]
for fa in factorList:
    ans=GRS(betaPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

for port,name in list(zip(portfolioList[1:],portnameList)):
    counter+=1
    df.loc[counter]=[name,"","","","","","",""]
    for fa in factorList[1:]:
            ans=GRS(port,fa)
            counter+=1
            df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  
counter+=1       
df.loc[counter]=['25 Size Prior2-12 portfolios',"","","","","","",""]
for fa in momfactorlist:
    ans=GRS(momPortXs,fa)
    counter+=1
    df.loc[counter]=[indexname(fa),ans[0],1-f.cdf(ans[0],ans[1],ans[2]),ans[3],ans[4],ans[5],ans[6],ans[7]]  

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [16]:
df.set_index('Model Factors',drop=True,inplace=True)
#print (df.to_string(index=False))

In [17]:
df

,GRS,p(GRS),A|ai|,A|ai|/A|ri|,Aai^2/Ari^2,As^2(ai)/Aasi^2,A(R^2)
Model Factors,,,,,,,
25 Size Beta Portfolio,,,,,,,
Mkt-RF,2.24749,0.000537194,0.232309,1.03477,0.99433,0.177227,0.750919
Mkt-RF SMB HML,1.58636,0.035797,0.10697,0.476473,0.201101,0.377368,0.885953
Mkt-RF SMB HML RMW,1.80318,0.0100953,0.0808836,0.360278,0.126707,0.580984,0.892267
Mkt-RF SMB HML CMA,1.47581,0.0646481,0.0927158,0.412982,0.164609,0.466788,0.886951
Mkt-RF SMB RMW CMA,1.8431,0.00788961,0.0733228,0.3266,0.103446,0.759627,0.887215
Mkt-RF SMB HML RMW CMA,1.85619,0.00727347,0.0734208,0.327037,0.103555,0.723679,0.894098
35 Size-NI Portffolio,,,,,,,
Mkt-RF SMB HML,4.25808,6.86118e-14,0.138916,0.723221,0.404626,0.180074,0.87091


### 2.3 The five factor model vs FF three factor model

In [18]:
RMWexposure=GRS(FF5[0][['RMW']],F2)
CMAexposure=GRS(FF5[0][['CMA']],F2)


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

In [19]:
RMWexposure
CMAexposure

(12.127891912400209,
 1,
 614,
 0.19782252018339455,
 -1.064376784549344,
 1.1328979394876004,
 0.08246044901480107,
 0.5325692604844768)

### 2.4 An equivalent five factor model

In [20]:
#Equivalent five factore model
HMLexposure=GRS(FF5[0][['HML']],F5)
HMLexposure

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directl

(0.0003076793848766597,
 1,
 613,
 0.0014805408045507479,
 -0.011397287209919813,
 0.00012989815574540175,
 3250.65773652803,
 0.5006616953708781)

In [21]:
model = sm.OLS(FF5[0][['HML']], sm.add_constant(F5))

In [22]:
model.fit().summary() #Constant is around 0.0016, does not really add value or means that nothing is lost in explanation of average returns 
#if we drop HML from the model

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    HML   R-squared:                       0.504
Model:                            OLS   Adj. R-squared:                  0.501
Method:                 Least Squares   F-statistic:                     155.7
Date:                Sat, 31 Oct 2020   Prob (F-statistic):           7.65e-92
Time:                        11:05:54   Log-Likelihood:                -1300.2
No. Observations:                 618   AIC:                             2610.
Df Residuals:                     613   BIC:                             2633.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0015      0.084     -0.018      0.986      -0.167       0.164
Mkt-RF         0.0171      0.021      0.830      0.407      -0.023       0.057
SMB            0.0268      0.029      0.932      0.352      -0.030       0.083
RMW            0.1514      0.039      3.868      0.000       0.075       0.228
CMA            1.0039      0.044     23.008      0.000       0.918       1.090
==============================================================================
Omnibus:                       42.454   Durbin-Watson:                   1.725
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              145.947
Skew:                          -0.206   Prob(JB):                     2.03e-32
Kurtosis:                       5.345   Cond. No.                         5.08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [23]:
##Calculate HMLO
HMLO=model.fit().params[0]+model.fit().resid
HMLO

Date
1963-07    0.243781
1963-08    1.945587
1963-09    0.093831
1963-10    1.723674
1963-11   -0.413395
             ...   
2014-08    0.134607
2014-09   -0.831468
2014-10   -1.701555
2014-11   -3.278496
2014-12    1.396545
Freq: M, Length: 618, dtype: float64

## 3. Market Beta

In [24]:
dt3=betaPortXs.mean()
dt3
dt3sd=betaPortXs.std()
dt3sd

SMALL LoBETA    4.403566
ME1 BETA2       5.092372
ME1 BETA3       5.946667
ME1 BETA4       6.514479
SMALL HiBETA    8.230605
ME2 BETA1       4.303275
ME2 BETA2       4.776218
ME2 BETA3       5.472472
ME2 BETA4       6.204383
ME2 BETA5       7.912692
ME3 BETA1       3.859098
ME3 BETA2       4.643507
ME3 BETA3       5.225233
ME3 BETA4       5.984497
ME3 BETA5       7.665540
ME4 BETA1       3.881337
ME4 BETA2       4.623011
ME4 BETA3       5.168256
ME4 BETA4       5.839759
ME4 BETA5       7.589291
BIG LoBETA      3.627947
ME5 BETA2       4.254481
ME5 BETA3       4.874660
ME5 BETA4       5.703102
BIG HiBETA      7.144848
dtype: float64

In [25]:

# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Mean,,,,,,SD,,
small,0.732704,0.89061,0.928205,0.971773,0.796989,,4.40357,5.09237,5.94667,6.51448,8.2306
2,0.70595,0.867878,0.950831,0.884945,0.715566,,4.30328,4.77622,5.47247,6.20438,7.91269
3,0.689102,0.862609,0.838617,0.804877,0.75331,,3.8591,4.64351,5.22523,5.9845,7.66554
4,0.669165,0.761869,0.734495,0.587422,0.752701,,3.88134,4.62301,5.16826,5.83976,7.58929
Big,0.488234,0.516946,0.495187,0.497284,0.404631,,3.62795,4.25448,4.87466,5.7031,7.14485


In [26]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[6].mean()
dt3b=betaPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,B/M,,,,,,OP,,
small,0.978333,1.02833,1.03208,1.03212,0.935283,,0.166656,0.157609,0.169664,0.79097,0.0452204
2,0.85808,0.85243,0.826256,0.803328,0.739945,,0.266445,0.260208,0.376776,0.341499,0.240984
3,0.81999,0.773257,0.72866,0.695218,0.661509,,0.272471,0.308223,0.337963,0.343918,0.327285
4,0.756989,0.686216,0.668542,0.6671,0.6423,,0.314529,0.382237,0.321304,0.342518,0.323477
Big,0.611213,0.477666,0.543125,0.532282,0.616422,,0.403892,0.467218,0.3724,0.366538,0.331539


In [27]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=betaPort[10].mean()
dt3b=betaPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior-Beta',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,Inv,,,,,,Prior-Beta,,
small,0.0975382,0.0872642,0.0964864,0.0950502,0.123259,,0.321012,0.836558,1.15124,1.5117,2.47946
2,0.110291,0.107712,0.122171,0.135044,0.177737,,0.380648,0.83935,1.15423,1.50747,2.39695
3,0.110965,0.115324,0.124179,0.14221,0.194609,,0.382849,0.838484,1.15048,1.49786,2.2957
4,0.0990663,0.110462,0.118131,0.12904,0.186805,,0.397009,0.840444,1.14688,1.49714,2.23278
Big,0.0906003,0.110098,0.133069,0.142191,0.181013,,0.42251,0.828333,1.1441,1.48018,2.0965


In [70]:
##Continue to the deeper regression on each port
def regMarket(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
    
        
    return (alpha,beta,tstata,tstatb)
def regCAPM(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        c.append(fit.params[2])
        tstatc.append(fit.tvalues[2])
        d.append(fit.params[3])
        tstatd.append(fit.tvalues[3])
        
    return (alpha,beta,tstata,tstatb,c,tstatc,d,tstatd)
def regFourFactor(R,F):
    alpha=[]
    beta=[]
    tstata=[]
    tstatb=[]
    c=[]
    tstatc=[]
    d=[]
    tstatd=[]
    e=[]
    tstate=[]
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        c.append(fit.params[2])
        tstatc.append(fit.tvalues[2])
        d.append(fit.params[3])
        tstatd.append(fit.params[3])
        e.append(fit.params[4])
        tstate.append(fit.tvalues[4])
    return (alpha,beta,tstata,tstatb,c,tstatc,d,tstatd,e,tstate)
            
def regFivefactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc)
def regSixfactor(R,F):
    alpha=[]
    beta=[]
    s=[]
    h=[]
    r=[]
    c=[]
    tstata=[]
    tstatb=[]
    tstats=[]
    tstath=[]
    tstatr=[]
    tstatc=[]
    m=[]
    tstatm=[]
       
    for p in R.columns:
        model = sm.OLS(R[:][p], sm.add_constant(F[:]))
        fit = model.fit()
        alpha.append(fit.params["const"])
        beta.append(fit.params[1])
        s.append(fit.params[2])
        h.append(fit.params[3])
        r.append(fit.params[4])
        c.append(fit.params[5])
        m.append(fit.params[6])
        tstata.append(fit.tvalues[0])
        tstatb.append(fit.tvalues[1])
        tstats.append(fit.tvalues[2])
        tstath.append(fit.tvalues[3])
        tstatr.append(fit.tvalues[4])
        tstatc.append(fit.tvalues[5])
        tstatm.append(fit.tvalues[6])
    return (alpha,beta,tstata,tstatb,s,tstats,h,tstath,r,tstatr,c,tstatc,m,tstatm)

In [31]:

result3a=regMarket(betaPortXs,F1)


In [32]:

# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4a

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.351051,0.434371,0.398533,0.359296,0.0463216,,3.03249,3.40958,2.65394,2.41163,0.2319
2,0.316107,0.410705,0.41022,0.271152,-0.0706051,,2.99124,3.91597,3.71733,2.17677,-0.450461
3,0.326199,0.389035,0.304046,0.194857,-0.0266911,,3.69774,4.64483,3.26773,1.80139,-0.191291
4,0.303572,0.278319,0.186251,-0.0292531,-0.025644,,3.43362,3.75464,2.44322,-0.330008,-0.191747
Big,0.150421,0.0673103,-0.0304781,-0.112867,-0.314429,,1.77696,1.0475,-0.47955,-1.42415,-2.31985
,,,b,,,,,,t(b),,
small,0.751918,0.898864,1.04354,1.20668,1.47894,,29.1106,31.6217,31.145,36.2995,33.1832
2,0.768053,0.900703,1.06509,1.20927,1.54888,,32.5732,38.4895,43.2564,43.5086,44.2885
3,0.714977,0.933017,1.05319,1.20184,1.53673,,36.3243,49.9254,50.7299,49.7952,49.36


In [33]:
## Five Factor regression with HML->HMLO
test=F2.copy()
test=test.drop(['HML'],axis=1)
test['HMLO']=HMLO
test['RMW']=F5['RMW']
test['CMA']=F5['CMA']

In [34]:
result3a=regFivefactor(betaPortXs,test) #For five factor regression

In [35]:
#List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High β',' ','Low β' , '2', '3' , '4','High β'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High β,,Low β,2,3,4,High β
,,,a,,,,,,t(a),,
small,0.0845882,0.0742892,0.0616118,0.0786149,0.00212861,,1.28086,1.15359,0.935961,1.29534,0.0255543
2,0.0295526,0.0338029,0.0606004,-0.0607696,-0.141909,,0.400096,0.61492,1.03969,-1.02492,-1.87993
3,0.0687904,0.0945314,-0.00352404,-0.103927,-0.0234189,,0.912724,1.60283,-0.0571499,-1.47658,-0.261093
4,0.0446856,-0.0248427,-0.0942255,-0.246741,0.0975095,,0.545652,-0.394512,-1.44778,-3.14011,0.881041
Big,-0.0826826,-0.0849706,-0.0892347,-0.0931499,-0.0554187,,-1.25587,-1.61133,-1.50813,-1.14736,-0.407307
,,,b,,,,,,t(b),,
small,0.663843,0.810016,0.895916,1.02965,1.14435,,41.2251,51.5852,55.8168,69.5779,56.3418
2,0.730095,0.86135,0.996999,1.09788,1.29452,,40.537,64.2612,70.1498,75.9382,70.3309
3,0.749647,0.922356,1.0192,1.13225,1.31169,,40.7918,64.138,67.7858,65.9745,59.9744


### 4.NI Net Share Issues

In [36]:
dt3=niPortXs.mean()
dt3
dt3sd=niPortXs.std()
dt3sd.head()

SMALL NegNI     5.667709
SMALL ZeroNI    5.705591
SMALL LoNI      6.362067
ME1 NI2         6.432492
ME1 NI3         6.857032
dtype: float64

In [37]:
# List of tuples
students = [ (' ',' ',' ','Mean',' ',' ',' ',' ',' ',' ',' ','SD',' ',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],dt3[5], dt3[6],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4],dt3sd[5], dt3sd[6]),
             (dt3[7], dt3[8], dt3[9] , dt3[10],dt3[11],dt3[12], dt3[13],' ',dt3sd[7], dt3sd[8], dt3sd[9] , dt3sd[10],dt3sd[11],dt3sd[12], dt3sd[13]),
             (dt3[14], dt3[15], dt3[16],dt3[17],dt3[18],dt3[19], dt3[20],' ',dt3sd[14], dt3sd[15], dt3sd[16] , dt3sd[17],dt3sd[18],dt3sd[19], dt3sd[20]),
             (dt3[21], dt3[22], dt3[23] , dt3[24],dt3[25],dt3[26], dt3[27],' ',dt3sd[21], dt3sd[22], dt3sd[23] , dt3sd[24],dt3sd[25],dt3sd[26], dt3sd[27]),
             (dt3[28], dt3[29], dt3[30] , dt3[31],dt3[32],dt3[33], dt3[34],' ',dt3sd[28], dt3sd[29], dt3sd[30] , dt3sd[31],dt3sd[32],dt3sd[33], dt3sd[34])]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a 

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Mean,,,,,,,,SD,,,
small,1.04094,0.763188,0.899782,0.950428,1.02083,0.698482,0.242586,,5.66771,5.70559,6.36207,6.43249,6.85703,7.12108,7.7379
2,0.909644,0.802531,0.914072,0.913302,0.850793,0.812677,0.306043,,5.30366,5.63293,5.83896,6.12738,6.32045,6.53041,7.07975
3,0.963984,0.751828,0.833877,0.929535,0.832749,0.723224,0.286675,,5.07838,5.43081,5.28207,5.62796,5.83469,6.01682,6.54849
4,0.941052,0.560273,0.653033,0.687051,0.820516,0.621508,0.363179,,4.91186,4.88524,5.07014,5.33553,5.45457,5.80948,6.1861
Big,0.615858,0.608861,0.490563,0.479228,0.590431,0.431733,0.177375,,4.17591,4.78653,4.24836,4.55064,4.85929,5.41809,5.12104


In [38]:
#6: B/M  10:OP 11:inv  8:NI
dt3a=niPort[6].mean()
dt3b=niPort[10].mean()
dt3b['BIG ZeroNI']=niPort[10]['BIG ZeroNI'].median()#Some error in the dataset. Already screenshot
dt3b.round(6)
# List of tuples
students = [ (' ',' ',' ','B/M',' ',' ',' ',' ',' ',' ',' ','OP',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3a

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,B/M,,,,,,,,OP,,,
small,1.04476,1.15296,1.00547,0.915588,0.853405,0.748871,0.682065,,0.605454,0.237369,0.213195,0.22797,0.211598,0.096845,-0.0908481
2,0.845488,0.981042,0.838411,0.770777,0.710302,0.660525,0.619349,,0.307616,0.43016,0.275123,0.280878,0.358919,0.251071,0.249917
3,0.763913,0.881798,0.808708,0.701248,0.657364,0.632834,0.624717,,0.331844,0.316228,0.301308,0.300526,0.320399,0.320402,0.280896
4,0.674106,0.858345,0.754801,0.649707,0.640819,0.641537,0.651967,,0.393329,0.289537,0.299805,0.302986,0.375733,0.304953,0.248567
Big,0.569372,0.730628,0.550846,0.487259,0.494469,0.580024,0.674095,,0.427204,0.2926,0.345649,0.36915,0.487236,0.332891,0.302154


In [61]:
#6: B/M  10:OP 11:inv  8:NI
dt3a=niPort[11].mean()
dt3b=niPort[8].mean()
# List of tuples
students = [ (' ',' ',' ','Inv',' ',' ',' ',' ',' ',' ',' ','NI',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,Inv,,,,,,,,NI,,,
small,0.0542214,0.0566117,0.0679545,0.0730845,0.0878421,0.131434,0.430096,,-5.04372,0,0.136619,0.52195,1.28355,3.83696,35.6197
2,0.0719739,0.0942916,0.0920633,0.101106,0.12428,0.160133,0.424702,,-4.51042,0,0.140013,0.517856,1.287,3.89883,33.8003
3,0.0714217,0.0971451,0.0870273,0.104879,0.129643,0.175289,0.39225,,-3.85821,0,0.134904,0.519177,1.28948,3.95296,28.6753
4,0.0691547,0.0709977,0.0814646,0.0986638,0.117902,0.164597,0.336587,,-3.37959,0,0.138889,0.515027,1.28062,3.94931,24.3706
Big,0.0779016,0.0914681,0.0881613,0.111262,0.126351,0.162822,0.317141,,-1.98569,0,0.126653,0.507801,1.2765,3.83819,22.4061


### Regression

In [40]:
result3a=regCAPM(niPortXs,F2)
dt3a=result3a[0]
dt3b=result3a[2]

In [41]:
# List of tuples
regcapmdata = [ (' ',' ',' ','a',' ',' ',' ',' ',' ',' ',' ','t(a)',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34])]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,a,,,,,,,,t(a),,,
small,0.19795,-0.0253023,0.00721058,0.074853,0.1798,-0.147634,-0.568226,,3.35567,-0.3144,0.099839,1.08944,2.44854,-1.95112,-6.00076
2,0.10917,0.0430992,0.0834105,0.0967504,0.0550864,0.0443945,-0.434978,,1.93727,0.380739,1.1954,1.36668,0.825192,0.66596,-5.70284
3,0.238145,0.0351879,0.0494813,0.183825,0.128262,0.0149848,-0.36701,,3.58052,0.293549,0.668565,2.61226,1.78685,0.203656,-4.49554
4,0.238651,-0.0138735,-0.0477649,0.0106066,0.215035,0.0106012,-0.279051,,3.49336,-0.110093,-0.628796,0.141303,2.93482,0.127767,-2.9526
Big,0.142391,0.161532,0.0682828,0.0612964,0.144908,0.0200295,-0.363271,,2.62951,1.15948,0.985571,0.955231,1.9982,0.233587,-4.27019


In [42]:
result3a=regFivefactor(niPortXs,test)  #test already include HMLO
dt3a=result3a[0] 
dt3b=result3a[2]
dt3c=result3a[6] 

dt3d=result3a[7] 
dt3e=result3a[8]
dt3f=result3a[9]
dt3g=result3a[10]
dt3h=result3a[11]

In [43]:
reg5FdataNI = [ (' ',' ',' ','a',' ',' ',' ',' ',' ',' ',' ','t(a)',' ',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],dt3a[5], dt3a[6],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4],dt3b[5], dt3b[6]),
             (dt3a[7], dt3a[8], dt3a[9] , dt3a[10],dt3a[11],dt3a[12], dt3a[13],' ',dt3b[7], dt3b[8], dt3b[9] , dt3b[10],dt3b[11],dt3b[12], dt3b[13]),
             (dt3a[14], dt3a[15], dt3a[16],dt3a[17],dt3a[18],dt3a[19], dt3a[20],' ',dt3b[14], dt3b[15], dt3b[16] , dt3b[17],dt3b[18],dt3b[19], dt3b[20]),
             (dt3a[21], dt3a[22], dt3a[23] , dt3a[24],dt3a[25],dt3a[26], dt3a[27],' ',dt3b[21], dt3b[22], dt3b[23] , dt3b[24],dt3b[25],dt3b[26], dt3b[27]),
             (dt3a[28], dt3a[29], dt3a[30] , dt3a[31],dt3a[32],dt3a[33], dt3a[34],' ',dt3b[28], dt3b[29], dt3b[30] , dt3b[31],dt3b[32],dt3b[33], dt3b[34]),
             (' ',' ',' ','h',' ',' ',' ',' ',' ',' ',' ','t(h)',' ',' ', ' '),
             (dt3c[0], dt3c[1], dt3c[2] , dt3c[3],dt3c[4],dt3c[5], dt3c[6],' ',dt3d[0], dt3d[1], dt3d[2] , dt3d[3],dt3d[4],dt3d[5], dt3d[6]),
             (dt3c[7], dt3c[8], dt3c[9] , dt3c[10],dt3c[11],dt3c[12], dt3c[13],' ',dt3d[7], dt3d[8], dt3d[9] , dt3d[10],dt3d[11],dt3d[12], dt3d[13]),
             (dt3c[14], dt3c[15], dt3c[16],dt3c[17],dt3c[18],dt3c[19], dt3c[20],' ',dt3d[14], dt3d[15], dt3d[16] , dt3d[17],dt3d[18],dt3d[19], dt3d[20]),
             (dt3c[21], dt3c[22], dt3c[23] , dt3c[24],dt3c[25],dt3c[26], dt3c[27],' ',dt3d[21], dt3d[22], dt3d[23] , dt3d[24],dt3d[25],dt3d[26], dt3d[27]),
             (dt3c[28], dt3c[29], dt3c[30] , dt3c[31],dt3c[32],dt3c[33], dt3c[34],' ',dt3d[28], dt3d[29], dt3d[30] , dt3d[31],dt3d[32],dt3d[33], dt3d[34]), 
             (' ',' ',' ','r',' ',' ',' ',' ',' ',' ',' ','t(r)',' ',' ', ' '),
             (dt3e[0], dt3e[1], dt3e[2] , dt3e[3],dt3e[4],dt3e[5], dt3e[6],' ',dt3f[0], dt3f[1], dt3f[2] , dt3f[3],dt3f[4],dt3f[5], dt3f[6]),
             (dt3e[7], dt3e[8], dt3e[9] , dt3e[10],dt3e[11],dt3e[12], dt3e[13],' ',dt3f[7], dt3f[8], dt3f[9] , dt3f[10],dt3f[11],dt3f[12], dt3f[13]),
             (dt3e[14], dt3e[15], dt3e[16],dt3e[17],dt3e[18],dt3e[19], dt3e[20],' ',dt3f[14], dt3f[15], dt3f[16] , dt3f[17],dt3f[18],dt3f[19], dt3f[20]),
             (dt3e[21], dt3e[22], dt3e[23] , dt3e[24],dt3e[25],dt3e[26], dt3e[27],' ',dt3f[21], dt3f[22], dt3f[23] , dt3f[24],dt3f[25],dt3f[26], dt3f[27]),
             (dt3e[28], dt3e[29], dt3e[30] , dt3e[31],dt3e[32],dt3e[33], dt3e[34],' ',dt3f[28], dt3f[29], dt3f[30] , dt3f[31],dt3f[32],dt3f[33], dt3f[34]),
             (' ',' ',' ','c',' ',' ',' ',' ',' ',' ',' ','t(c)',' ',' ', ' '),
             (dt3g[0], dt3g[1], dt3g[2] , dt3g[3],dt3g[4],dt3g[5], dt3g[6],' ',dt3h[0], dt3h[1], dt3h[2] , dt3h[3],dt3h[4],dt3h[5], dt3h[6]),
             (dt3g[7], dt3g[8], dt3g[9] , dt3g[10],dt3g[11],dt3g[12], dt3g[13],' ',dt3h[7], dt3h[8], dt3h[9] , dt3h[10],dt3h[11],dt3h[12], dt3h[13]),
             (dt3g[14], dt3g[15], dt3g[16],dt3g[17],dt3g[18],dt3g[19], dt3g[20],' ',dt3h[14], dt3h[15], dt3h[16] , dt3h[17],dt3h[18],dt3h[19], dt3h[20]),
             (dt3g[21], dt3g[22], dt3g[23] , dt3g[24],dt3g[25],dt3g[26], dt3g[27],' ',dt3h[21], dt3h[22], dt3h[23] , dt3h[24],dt3h[25],dt3h[26], dt3h[27]),
             (dt3g[28], dt3g[29], dt3g[30] , dt3g[31],dt3g[32],dt3g[33], dt3g[34],' ',dt3h[28], dt3h[29], dt3h[30] , dt3h[31],dt3h[32],dt3h[33], dt3h[34])]


# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5FdataNI, columns =['Neg','Zero','Low' , '2', '3' , '4','High ',' ','Neg','Zero','Low', '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Neg,Zero,Low,2,3,4,High,,Neg,Zero,Low,2,3,4,High
,,,,a,,,,,,,,t(a),,,
small,0.102639,-0.0654091,-0.0150017,0.0541348,0.246432,-0.0333932,-0.346904,,1.7536,-0.790391,-0.201922,0.769763,3.40036,-0.447557,-4.10328
2,-0.0329725,-0.0547101,0.00123907,0.00936528,0.043127,0.139615,-0.241462,,-0.631363,-0.479355,0.0178493,0.131,0.626697,2.09401,-3.40527
3,0.0689711,-0.0297684,-0.0507961,0.123525,0.123042,0.0934036,-0.120853,,1.15161,-0.242075,-0.690576,1.71952,1.66209,1.25071,-1.65098
4,0.0657875,-0.214597,-0.198591,-0.102599,0.225135,0.156458,0.00490835,,1.03344,-1.71291,-2.67952,-1.36798,2.97911,1.9503,0.0584194
Big,0.0110428,-0.0144096,-0.0803221,0.0370022,0.156864,0.196794,-0.197,,0.215422,-0.102719,-1.21143,0.566228,2.10594,2.37651,-2.39906
,,,,h,,,,,,,,t(h),,,
small,0.263871,0.206959,0.218262,0.128063,-0.0289659,-0.0109865,-0.203107,,9.42211,5.22666,6.13983,3.80575,-0.835317,-0.307741,-5.02091
2,0.241817,0.306279,0.25423,0.0442863,0.0356002,-0.0132058,-0.126627,,9.67722,5.60845,7.65402,1.29466,1.08118,-0.413952,-3.7322
3,0.19674,0.225765,0.33546,0.105153,0.00430435,0.0566944,-0.0516659,,6.86544,3.83697,9.53143,3.05921,0.121519,1.5866,-1.47512


## 5.Volatility (r-Var portfolios)

In [44]:
dt3=rVarPortXs.mean()
dt3
dt3sd=rVarPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,1.00546,1.19155,1.06144,0.828107,-0.200419,,4.19199,5.63767,6.4858,7.52526,8.99671
2,0.912336,1.03527,1.06947,0.958722,0.182537,,4.12099,5.29515,5.96254,6.81887,8.42494
3,0.746026,0.914126,0.929021,0.928187,0.346236,,3.79937,4.85446,5.42241,6.20657,7.7813
4,0.716318,0.759244,0.747809,0.775418,0.441043,,3.83747,4.52668,5.18602,5.73831,7.40797
Big,0.467783,0.509183,0.505353,0.483248,0.465846,,3.65507,4.15277,4.56382,5.02592,6.42179


In [45]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=rVarPort[6].mean()
dt3b=rVarPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 


,Low,2,3,4,High,,Low,2,3,4,High
,,,B/M,,,,,,OP,,
small,0.990221,0.939926,0.90804,0.88443,0.924262,,0.243425,0.316652,0.359635,0.494062,0.00149951
2,0.860785,0.810927,0.777557,0.750094,0.712719,,0.329109,0.338307,0.317774,0.299829,0.204159
3,0.808653,0.73995,0.706088,0.685962,0.657467,,0.300579,0.327998,0.338076,0.346929,0.292489
4,0.769833,0.698774,0.663353,0.648226,0.62113,,0.310612,0.368649,0.39025,0.335545,0.362641
Big,0.566306,0.552384,0.551133,0.556377,0.530313,,0.379072,0.392288,0.438271,0.395855,0.428283


In [46]:
#B/M=6/7 OP= 9 INV=10 PRIOR B= 8
dt3a=rVarPort[10].mean()
dt3b=rVarPort[8].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','RVar',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low,2,3,4,High,,Low,2,3,4,High
,,,Inv,,,,,,RVar,,
small,0.102984,0.129102,0.158897,0.186667,0.190915,,2.04407,4.64588,7.53946,12.9408,41.1696
2,0.0958445,0.122445,0.150456,0.18777,0.253332,,1.4106,2.89909,4.39882,6.66403,16.8149
3,0.0960892,0.113925,0.133905,0.165694,0.249887,,1.14705,2.23145,3.36032,5.04259,12.5337
4,0.0902155,0.103545,0.117161,0.1382,0.218164,,1.03548,1.84156,2.68547,3.94982,9.70507
Big,0.0989898,0.105989,0.115588,0.135087,0.185336,,0.829931,1.40178,1.94256,2.75354,5.8401


### Regressions

In [47]:
result3a=regCAPM(rVarPortXs,F2)
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.335085,0.336508,0.123107,-0.156305,-1.22561,,5.07113,4.6968,1.61441,-1.57464,-7.60788
2,0.260976,0.218146,0.202359,0.0428552,-0.718844,,4.06935,3.24413,2.79343,0.55942,-7.11211
3,0.150752,0.179642,0.140462,0.0951944,-0.438717,,2.27037,2.62002,1.93991,1.22142,-4.62204
4,0.151526,0.118066,0.0419822,0.0452043,-0.276767,,2.01183,1.62027,0.578129,0.583487,-2.70459
Big,0.0767754,0.0980629,-0.000360191,-0.0588735,-0.0594252,,1.34157,1.82091,-0.00647827,-1.03105,-0.637417


In [48]:
result3a=regFivefactor(rVarPortXs,test) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.216483,0.193872,0.0619423,-0.0696785,-0.841267,,3.39419,2.87052,0.805166,-0.687507,-5.6194
2,0.113202,0.0364758,0.0311077,-0.0671795,-0.456168,,1.89004,0.617582,0.4754,-0.91843,-4.88227
3,0.00456649,0.0167732,-0.0382736,-0.0342419,-0.215915,,0.072323,0.275869,-0.596514,-0.462832,-2.41002
4,-0.00399319,-0.062285,-0.126847,-0.0714166,-0.0337517,,-0.0550489,-0.940533,-1.89081,-0.938822,-0.353923
Big,-0.0040161,-0.0386756,-0.117076,-0.0496792,0.159042,,-0.0706691,-0.770674,-2.20813,-0.844642,1.80003
,,,b,,,,,,t(b),,
small,0.721886,0.98628,1.10009,1.15155,1.11467,,46.418,59.8895,58.6449,46.5978,30.5356
2,0.783081,1.00935,1.12415,1.24459,1.25705,,53.62,70.0867,70.4558,69.7816,55.1764
3,0.787123,0.999794,1.10232,1.20585,1.2482,,51.1259,67.4378,70.4586,66.8439,57.1383


## 6. Accrual 

In [49]:
dt3=acPortXs.mean()
dt3
dt3sd=acPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.878597,0.917432,0.831241,0.960878,0.585963,,7.11024,6.55514,6.20445,6.46699,7.23228
2,0.823225,0.879569,0.809161,0.737645,0.630895,,6.55443,5.80423,5.87354,6.16174,6.80097
3,0.871748,0.784932,0.836005,0.789624,0.599852,,6.20291,5.48552,5.21693,5.66145,6.62647
4,0.739392,0.691331,0.65619,0.766102,0.705468,,5.69058,5.11223,4.97881,5.20171,6.20464
Big,0.622578,0.526145,0.503253,0.491924,0.271217,,5.08196,4.22718,4.05099,4.54919,5.18815


In [50]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=acPort[6].mean()
dt3b=acPort[10].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 



,Low,2,3,4,High,,Low,2,3,4,High
,,,B/M,,,,,,OP,,
small,0.908704,0.968738,0.97401,0.904383,0.756935,,-0.0357155,0.156483,0.161599,0.186216,0.589523
2,0.737796,0.827725,0.771076,0.716968,0.616757,,0.233077,0.229326,0.237318,0.282066,0.381258
3,0.689111,0.740017,0.743683,0.655928,0.547557,,0.340369,0.25868,0.264691,0.287973,0.398697
4,0.632291,0.717056,0.699828,0.602935,0.491043,,0.380551,0.282921,0.283056,0.311904,0.489732
Big,0.505491,0.561972,0.535749,0.451413,0.396846,,0.450749,0.351935,0.348617,0.382523,0.734309


In [51]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=acPort[11].mean()
dt3b=acPort[9].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','AC',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

,Low,2,3,4,High,,Low,2,3,4,High
,,,Inv,,,,,,AC,,
small,0.0804772,0.127625,0.141468,0.157948,0.259744,,-63.2704,-2.42348,1.73194,6.31314,59.3321
2,0.127012,0.141072,0.155298,0.164447,0.268954,,-42.3276,-2.34412,1.72032,6.24977,46.1542
3,0.132308,0.138892,0.138152,0.160333,0.260059,,-94.7273,-2.32928,1.72323,6.17756,40.5509
4,0.111112,0.114342,0.123366,0.142237,0.222368,,-24.1713,-2.3046,1.66796,6.10232,30.4538
Big,0.126785,0.110751,0.110372,0.123991,0.175971,,-17.0669,-2.28157,1.63383,5.90442,30.2555


### Regressions

### Panel A: Regression Intercepts

#### Three Factor 

In [52]:
result3a=regCAPM(acPortXs,F2) #MKT,SMB,HML
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.012708,0.127216,0.0640214,0.162426,-0.284273,,0.16295,1.67269,0.933138,2.53375,-4.13481
2,0.0104312,0.145532,0.0817634,-0.0426033,-0.161256,,0.170126,2.33417,1.29611,-0.699092,-2.61701
3,0.15455,0.13623,0.184954,0.0730285,-0.170946,,1.98399,1.95333,2.8122,1.13088,-2.0845
4,0.0958197,0.0678385,0.0192269,0.166908,0.0273654,,1.20518,0.999341,0.289028,2.51195,0.331461
Big,0.210637,0.166694,0.112487,0.0663168,-0.156739,,2.67263,3.01628,2.0278,1.0732,-1.78205


#### Four Factor

In [53]:
result3a=regFourFactor(acPortXs,F4) #MKT,SMB,HML,CMA
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.0337409,0.0758003,0.0212661,0.14539,-0.278535,,-0.434489,1.00527,0.311728,2.25067,-4.00981
2,-0.00818841,0.122652,0.0483324,-0.0450529,-0.134141,,-0.132646,1.95737,0.767022,-0.731545,-2.17143
3,0.160174,0.104292,0.159631,0.087001,-0.105509,,2.03495,1.49259,2.41641,1.33563,-1.3078
4,0.0685015,0.0242804,0.0170637,0.146341,0.0615383,,0.856684,0.360075,0.253817,2.18787,0.74279
Big,0.207718,0.155707,0.0807678,0.0908093,-0.100295,,2.60796,2.79234,1.46048,1.46358,-1.14795


#### Five Factor

In [54]:
result3a=regCAPM(acPortXs,test) #MKT,SMB,HMLO,RMW,CMA
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.109706,0.25631,0.142994,0.225833,-0.183829,,1.49012,3.79722,2.19924,3.53682,-2.6953
2,0.038016,0.138508,0.0943377,-0.0647125,-0.153933,,0.610174,2.16804,1.48253,-1.03142,-2.44596
3,0.192581,0.161949,0.188896,0.0444621,-0.197201,,2.4049,2.3022,2.81211,0.674452,-2.45928
4,0.062873,0.0857283,-0.0078923,0.137368,0.037985,,0.770301,1.26674,-0.115317,2.01247,0.449856
Big,0.230305,0.207206,0.0189557,0.0257335,-0.183281,,2.83707,3.70418,0.344549,0.415594,-2.09334


### Panel B: Regression slopes

#### 3 Factors

In [55]:
result3a=regCAPM(acPortXs,F2) #MKT,SMB,HML
# List of tuples
regcapmdata = [ ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,h,,,,,,t(h),,
small,-0.00301965,-0.0117028,0.0223723,0.00912001,-0.0425126,,-0.107257,-0.426243,0.903288,0.394092,-1.7129
2,-0.00186798,0.0436358,0.00975934,0.0385378,-0.112577,,-0.0843926,1.9387,0.428546,1.75175,-5.06097
3,-0.0602873,-0.0232783,0.0446068,0.0664714,-0.0648043,,-2.14383,-0.924588,1.87879,2.85136,-2.18898
4,-0.0455613,0.0755172,0.142259,-0.0227324,-0.114319,,-1.58741,3.08161,5.92387,-0.947706,-3.8357
Big,-0.210705,-0.101261,0.00114792,-0.0841514,-0.200001,,-7.40583,-5.0756,0.0573233,-3.77235,-6.29896


#### 4 Factors

In [56]:
result3a=regFourFactor(acPortXs,F4) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24])]
            


# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,h,,,,,,t(h),,
small,-0.111033,-0.131266,-0.0770516,-0.0304946,-0.0291706,,-0.111033,-0.131266,-0.0770516,-0.0304946,-0.0291706
2,-0.0451663,-0.0095704,-0.0679819,0.0328416,-0.049524,,-0.0451663,-0.0095704,-0.0679819,0.0328416,-0.049524
3,-0.0472096,-0.0975476,-0.0142785,0.0989632,0.0873648,,-0.0472096,-0.0975476,-0.0142785,0.0989632,0.0873648
4,-0.109088,-0.0257736,0.137229,-0.0705582,-0.0348527,,-0.109088,-0.0257736,0.137229,-0.0705582,-0.0348527
Big,-0.217495,-0.126811,-0.0726124,-0.0271961,-0.0687447,,-0.217495,-0.126811,-0.0726124,-0.0271961,-0.0687447
,,,c,,,,,,t(c),,
small,0.234801,0.259908,0.216129,0.0861149,-0.029003,,4.29785,4.89963,4.5033,1.89489,-0.593495
2,0.0941226,0.115661,0.168995,0.0123825,-0.137066,,2.16729,2.62368,3.81218,0.285797,-3.15386
3,-0.0284285,0.161448,0.128006,-0.0706312,-0.330788,,-0.513387,3.28436,2.75431,-1.54131,-5.82816


#### 5 Factors

In [57]:
result3a=regFivefactor(acPortXs,test) #For five factor regression
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'b' , ' ',' ',' ',' ',' ','t(b)',' ', ' '),
             (result3a[1][0], result3a[1][1], result3a[1][2] , result3a[1][3],result3a[1][4],' ',result3a[3][0], result3a[3][1], result3a[3][2] , result3a[3][3],result3a[3][4]) ,
             (result3a[1][5], result3a[1][6], result3a[1][7] , result3a[1][8],result3a[1][9],' ',result3a[3][5], result3a[3][6], result3a[3][7] , result3a[3][8],result3a[3][9]) ,
             (result3a[1][10], result3a[1][11], result3a[1][12] , result3a[1][13],result3a[1][14],' ',result3a[3][10], result3a[3][11], result3a[3][12] , result3a[3][13],result3a[3][14]) ,
             (result3a[1][15], result3a[1][16], result3a[1][17] , result3a[1][18],result3a[1][19],' ',result3a[3][15], result3a[3][16], result3a[3][17] , result3a[3][18],result3a[3][19]) ,
             (result3a[1][20], result3a[1][21], result3a[1][22] , result3a[1][23],result3a[1][24],' ',result3a[3][20], result3a[3][21], result3a[3][22] , result3a[3][23],result3a[3][24]),
             ('', '  ' , 's' , ' ',' ',' ',' ',' ','t(s)',' ', ' '),
             (result3a[4][0], result3a[4][1], result3a[4][2] , result3a[4][3],result3a[4][4],' ',result3a[5][0], result3a[5][1], result3a[5][2] , result3a[5][3],result3a[5][4]),
             (result3a[4][5], result3a[4][6], result3a[4][7] , result3a[4][8],result3a[4][9],' ',result3a[5][5], result3a[5][6], result3a[5][7] , result3a[5][8],result3a[5][9]),
             (result3a[4][10], result3a[4][11], result3a[4][12] , result3a[4][13],result3a[4][14],' ',result3a[5][10], result3a[5][11], result3a[5][12] , result3a[5][13],result3a[5][14]),
             (result3a[4][15], result3a[4][16], result3a[4][17] , result3a[4][18],result3a[4][19],' ',result3a[5][15], result3a[5][16], result3a[5][17] , result3a[5][18],result3a[5][19]),
             (result3a[4][20], result3a[4][21], result3a[4][22] , result3a[4][23],result3a[4][24],' ',result3a[5][20], result3a[5][21], result3a[5][22] , result3a[5][23],result3a[5][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,0.109706,0.25631,0.142994,0.225833,-0.183829,,1.49012,3.79722,2.19924,3.53682,-2.6953
2,0.038016,0.138508,0.0943377,-0.0647125,-0.153933,,0.610174,2.16804,1.48253,-1.03142,-2.44596
3,0.192581,0.161949,0.188896,0.0444621,-0.197201,,2.4049,2.3022,2.81211,0.674452,-2.45928
4,0.062873,0.0857283,-0.0078923,0.137368,0.037985,,0.770301,1.26674,-0.115317,2.01247,0.449856
Big,0.230305,0.207206,0.0189557,0.0257335,-0.183281,,2.83707,3.70418,0.344549,0.415594,-2.09334
,,,b,,,,,,t(b),,
small,1.02139,0.925426,0.902346,0.936335,1.02633,,56.8971,56.2272,56.9156,60.1398,61.7139
2,1.10409,1.00995,0.967524,1.01471,1.1093,,72.6766,64.8331,62.357,66.3269,72.2887
3,1.11209,0.990178,0.934631,1.06545,1.15989,,56.9546,57.7276,57.063,66.2825,59.3228


## 7.Momentum

In [65]:
dt3=momPortXs.mean()
dt3
dt3sd=momPortXs.std()
dt3sd
# List of tuples
students = [ (' ', '  ' , 'Mean' , ' ',' ',' ',' ',' ','SD',' ', ' '),
             (dt3[0], dt3[1], dt3[2] , dt3[3],dt3[4],' ',dt3sd[0], dt3sd[1], dt3sd[2] , dt3sd[3],dt3sd[4]) ,
             (dt3[5], dt3[6], dt3[7] , dt3[8],dt3[9],' ',dt3sd[5], dt3sd[6], dt3sd[7] , dt3sd[8],dt3sd[9]) ,
             (dt3[10], dt3[11], dt3[12] , dt3[13],dt3[14],' ',dt3sd[10], dt3sd[11], dt3sd[12] , dt3sd[13],dt3sd[14]) ,
             (dt3[15], dt3[16], dt3[17] , dt3[18],dt3[19],' ',dt3sd[15], dt3sd[16], dt3sd[17] , dt3sd[18],dt3sd[19]) ,
             (dt3[20], dt3[21], dt3[22] , dt3[23],dt3[24],' ',dt3sd[20], dt3sd[21], dt3sd[22] , dt3sd[23],dt3sd[24]) ]
# Create DataFrame object from a list of tuples
table3 = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3#Pnael a )

,Low,2,3,4,High,,Low,2,3,4,High
,,,Mean,,,,,,SD,,
small,0.0316667,0.674191,0.917848,1.04793,1.37927,,8.02271,5.87967,5.43549,5.50212,6.78006
2,0.13733,0.662557,0.811505,1.0096,1.22183,,7.83611,5.87228,5.26706,5.40386,6.7291
3,0.266052,0.613026,0.712411,0.781537,1.18249,,7.36034,5.5171,5.06464,5.00145,6.31417
4,0.198706,0.595453,0.650566,0.794094,1.04294,,7.26125,5.51979,4.87481,4.794,5.89273
Big,0.165356,0.45856,0.398236,0.549563,0.783204,,6.78976,4.88235,4.37073,4.31312,5.26569


In [69]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=momPort[6].mean()
dt3b=momPort[10].mean()
# List of tuples
students = [ (' ', '  ' , 'B/M' , ' ',' ',' ',' ',' ','OP',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3a = pd.DataFrame(students, columns = ['Low ' , '2', '3' , '4','High ',' ','Low ' , '2', '3' , '4','High '], index=['','small', '2', '3' , '4' , 'Big'])
table3a#Pnael a 




KeyError: 10

In [71]:
#B/M=6/7 OP= 10 INV=11 AC= 9
dt3a=momPort[11].mean()
dt3b=momPort[6].mean()
# List of tuples
students = [ (' ', '  ' , 'Inv' , ' ',' ',' ',' ',' ','Prior2-12',' ', ' '),
             (dt3a[0], dt3a[1], dt3a[2] , dt3a[3],dt3a[4],' ',dt3b[0], dt3b[1], dt3b[2] , dt3b[3],dt3b[4]) ,
             (dt3a[5], dt3a[6], dt3a[7] , dt3a[8],dt3a[9],' ',dt3b[5], dt3b[6], dt3b[7] , dt3b[8],dt3b[9]) ,
             (dt3a[10], dt3a[11], dt3a[12] , dt3a[13],dt3a[14],' ',dt3b[10], dt3b[11], dt3b[12] , dt3b[13],dt3b[14]) ,
             (dt3a[15], dt3a[16], dt3a[17] , dt3a[18],dt3a[19],' ',dt3b[15],dt3b[16], dt3b[17] , dt3b[18],dt3b[19]) ,
             (dt3a[20], dt3a[21], dt3a[22] , dt3a[23],dt3a[24],' ',dt3b[20], dt3b[21], dt3b[22] , dt3b[23],dt3b[24]) ]
# Create DataFrame object from a list of tuples
table3b = pd.DataFrame(students, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table3b#Pnael a 

KeyError: 11

### Regression 

#### 5 Factors

In [87]:
result3a=regFivefactor(momPortXs,F6)
# List of tuples
regcapmdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ]
# Create DataFrame object from a list of tuples
table4a = pd.DataFrame(regcapmdata, columns = ['Low' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big'])
table4a

,Low,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.710415,-0.196751,0.0389543,0.21565,0.622021,,-4.67743,-2.5001,0.618971,3.28729,6.4879
2,-0.620486,-0.209463,-0.0451354,0.153329,0.510896,,-4.40819,-2.56582,-0.787829,2.85371,5.69729
3,-0.372221,-0.18977,-0.126272,-0.0701448,0.510169,,-2.41478,-2.24751,-1.98103,-1.14019,5.26529
4,-0.418624,-0.210245,-0.147179,0.0115724,0.429017,,-2.50235,-2.16969,-2.18776,0.184504,4.01538
Big,-0.374856,-0.132754,-0.183083,-0.0427991,0.317862,,-2.28005,-1.31259,-2.84764,-0.673329,2.9555


#### 6 Factors

In [93]:
momtest=test.copy()
momtest['Mom']=FF6['Mom   ']

In [94]:
momtest

,Mkt-RF,SMB,HMLO,RMW,CMA,Mom
Date,,,,,,
1963-07,-0.39,-0.47,0.243781,0.66,-1.15,1.00
1963-08,5.07,-0.79,1.945587,0.40,-0.40,1.03
1963-09,-1.57,-0.48,0.093831,-0.76,0.24,0.16
1963-10,2.53,-1.29,1.723674,2.75,-2.24,3.14
1963-11,-0.85,-0.84,-0.413395,-0.45,2.22,-0.75
...,...,...,...,...,...,...
2014-08,4.24,0.30,0.134607,-0.61,-0.70,0.79
2014-09,-1.97,-3.75,-0.831468,1.24,-0.53,0.44
2014-10,2.52,3.80,-1.701555,-0.48,-0.19,-0.04


In [96]:
result3a=regSixfactor(momPortXs,momtest)
# List of tuples
reg5Fdata = [ ('', '  ' , 'a' , ' ',' ',' ',' ',' ','t(a)',' ', ' '),
             (result3a[0][0], result3a[0][1], result3a[0][2] , result3a[0][3],result3a[0][4],' ',result3a[2][0], result3a[2][1], result3a[2][2] , result3a[2][3],result3a[2][4]) ,
             (result3a[0][5], result3a[0][6], result3a[0][7] , result3a[0][8],result3a[0][9],' ',result3a[2][5], result3a[2][6], result3a[2][7] , result3a[2][8],result3a[2][9]) ,
             (result3a[0][10], result3a[0][11], result3a[0][12] , result3a[0][13],result3a[0][14],' ',result3a[2][10], result3a[2][11], result3a[2][12] , result3a[2][13],result3a[2][14]) ,
             (result3a[0][15], result3a[0][16], result3a[0][17] , result3a[0][18],result3a[0][19],' ',result3a[2][15], result3a[2][16], result3a[2][17] , result3a[2][18],result3a[2][19]) ,
             (result3a[0][20], result3a[0][21], result3a[0][22] , result3a[0][23],result3a[0][24],' ',result3a[2][20], result3a[2][21], result3a[2][22] , result3a[2][23],result3a[2][24]),
             ('', '  ' , 'h' , ' ',' ',' ',' ',' ','t(h)',' ', ' '),
             (result3a[6][0], result3a[6][1], result3a[6][2] , result3a[6][3],result3a[6][4],' ',result3a[7][0], result3a[7][1], result3a[7][2] , result3a[7][3],result3a[7][4]) ,
             (result3a[6][5], result3a[6][6], result3a[6][7] , result3a[6][8],result3a[6][9],' ',result3a[7][5], result3a[7][6], result3a[7][7] , result3a[7][8],result3a[7][9]) ,
             (result3a[6][10], result3a[6][11], result3a[6][12] , result3a[6][13],result3a[6][14],' ',result3a[7][10], result3a[7][11], result3a[7][12] , result3a[7][13],result3a[7][14]) ,
             (result3a[6][15], result3a[6][16], result3a[6][17] , result3a[6][18],result3a[6][19],' ',result3a[7][15], result3a[7][16], result3a[7][17] , result3a[7][18],result3a[7][19]) ,
             (result3a[6][20], result3a[6][21], result3a[6][22] , result3a[6][23],result3a[6][24],' ',result3a[7][20], result3a[7][21], result3a[7][22] , result3a[7][23],result3a[7][24]), 
             ('', '  ' , 'r' , ' ',' ',' ',' ',' ','t(r)',' ', ' '),
             (result3a[8][0], result3a[8][1], result3a[8][2] , result3a[8][3],result3a[8][4],' ',result3a[9][0], result3a[9][1], result3a[9][2] , result3a[9][3],result3a[9][4]) ,
             (result3a[8][5], result3a[8][6], result3a[8][7] , result3a[8][8],result3a[8][9],' ',result3a[9][5], result3a[9][6], result3a[9][7] , result3a[9][8],result3a[9][9]) ,
             (result3a[8][10], result3a[8][11], result3a[8][12] , result3a[8][13],result3a[8][14],' ',result3a[9][10], result3a[9][11], result3a[9][12] , result3a[9][13],result3a[9][14]) ,
             (result3a[8][15], result3a[8][16], result3a[8][17] , result3a[8][18],result3a[8][19],' ',result3a[9][15], result3a[9][16], result3a[9][17] , result3a[9][18],result3a[9][19]) ,
             (result3a[8][20], result3a[8][21], result3a[8][22] , result3a[8][23],result3a[8][24],' ',result3a[9][20], result3a[9][21], result3a[9][22] , result3a[9][23],result3a[9][24]),
             ('', '  ' , 'c' , ' ',' ',' ',' ',' ','t(c)',' ', ' '),
             (result3a[10][0], result3a[10][1], result3a[10][2] , result3a[10][3],result3a[10][4],' ',result3a[11][0], result3a[11][1], result3a[11][2] , result3a[11][3],result3a[11][4]) ,
             (result3a[10][5], result3a[10][6], result3a[10][7] , result3a[10][8],result3a[10][9],' ',result3a[11][5], result3a[11][6], result3a[11][7] , result3a[11][8],result3a[11][9]) ,
             (result3a[10][10], result3a[10][11], result3a[10][12] , result3a[10][13],result3a[10][14],' ',result3a[11][10], result3a[11][11], result3a[11][12] , result3a[11][13],result3a[11][14]) ,
             (result3a[10][15], result3a[10][16], result3a[10][17] , result3a[10][18],result3a[10][19],' ',result3a[11][15], result3a[11][16], result3a[11][17] , result3a[11][18],result3a[11][19]) ,
             (result3a[10][20], result3a[10][21], result3a[10][22] , result3a[10][23],result3a[10][24],' ',result3a[11][20], result3a[11][21], result3a[11][22] , result3a[11][23],result3a[11][24]),
             ('', '  ' , 'm' , ' ',' ',' ',' ',' ','t(m)',' ', ' '),
             (result3a[12][0], result3a[12][1], result3a[12][2] , result3a[12][3],result3a[12][4],' ',result3a[13][0], result3a[13][1], result3a[13][2] , result3a[13][3],result3a[13][4]) ,
             (result3a[12][5], result3a[12][6], result3a[12][7] , result3a[12][8],result3a[12][9],' ',result3a[13][5], result3a[13][6], result3a[13][7] , result3a[13][8],result3a[13][9]) ,
             (result3a[12][10], result3a[12][11], result3a[12][12] , result3a[12][13],result3a[12][14],' ',result3a[13][10], result3a[13][11], result3a[13][12] , result3a[13][13],result3a[13][14]) ,
             (result3a[12][15], result3a[12][16], result3a[12][17] , result3a[12][18],result3a[12][19],' ',result3a[13][15], result3a[13][16], result3a[13][17] , result3a[13][18],result3a[13][19]) ,
             (result3a[12][20], result3a[12][21], result3a[12][22] , result3a[12][23],result3a[12][24],' ',result3a[13][20], result3a[13][21], result3a[13][22] , result3a[13][23],result3a[3][24])]



# Create DataFrame object from a list of tuples
table4b = pd.DataFrame(reg5Fdata, columns = ['Low β' , '2', '3' , '4','High',' ','Low' , '2', '3' , '4','High'], index=['','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big','','small', '2', '3' , '4' , 'Big'])
table4b

,Low β,2,3,4,High,,Low,2,3,4,High
,,,a,,,,,,t(a),,
small,-0.223571,-0.00414898,0.104943,0.165405,0.401643,,-2.25879,-0.0640638,1.69709,2.52684,4.94109
2,-0.115456,0.0294727,0.00900102,0.104125,0.244224,,-1.57411,0.498088,0.158899,1.95679,3.81725
3,0.156935,0.0528224,-0.0128808,-0.128611,0.210852,,1.75722,0.848397,-0.22,-2.11498,3.18161
4,0.146316,0.0805627,-0.0282958,-0.0320999,0.104175,,1.46148,1.17644,-0.457236,-0.511357,1.3994
Big,0.163087,0.185128,-0.100691,-0.152508,-0.029105,,1.56989,2.73463,-1.62099,-2.59529,-0.419154
,,,h,,,,,,t(h),,
small,0.107805,0.276784,0.28567,0.213284,0.0533608,,2.24279,8.80037,9.51273,6.70931,1.35174
2,0.0584699,0.184804,0.239543,0.21551,0.0199253,,1.6415,6.43113,8.70768,8.33965,0.641292
3,0.0380408,0.186476,0.245276,0.273185,-0.0139824,,0.877091,6.16726,8.62633,9.25074,-0.434449
